In [1]:
import numpy as np
from scipy.sparse.linalg import eigsh
from tqdm import tqdm

import netket.operator as nkop
from netket.graph import Hypercube
from netket.hilbert import Spin
from netket.operator import Ising

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
SIDE_LENGTH = 3
DIMENSIONS = 2
PERIODIC_BOUNDARIES = True
J_COUPLING = -1.0

graph = Hypercube(length=SIDE_LENGTH, n_dim=DIMENSIONS, pbc=PERIODIC_BOUNDARIES)
hilbert = Spin(s=0.5, N=graph.n_nodes)

print(f"Total sites: {graph.n_nodes}")


Total sites: 9


In [3]:
def tfim_hamiltonian(h):
    return Ising(hilbert, graph, h=h, J=J_COUPLING)


def get_ground_state(hamiltonian):
    sp_mat = hamiltonian.to_sparse()
    vals, vecs = eigsh(sp_mat, k=1, which="SA")
    psi = vecs[:, 0]

    # Gauge fixing: essential for smooth surface
    idx = np.argmax(np.abs(psi))
    phase = np.angle(psi[idx])
    psi = psi * np.exp(-1j * phase)

    return psi


In [4]:
h_values = np.linspace(1.0, 4.0, 25)

ground_states = {}

print("Computing ground states...")
for h in tqdm(h_values):
    ham = tfim_hamiltonian(h)
    ground_states[h] = get_ground_state(ham)

print("Done.")


Computing ground states...


100%|██████████| 25/25 [00:00<00:00, 36.16it/s]

Done.


In [5]:
F = np.zeros((len(h_values), len(h_values)))

for i, h1 in enumerate(h_values):
    psi1 = ground_states[h1]
    for j, h2 in enumerate(h_values):
        psi2 = ground_states[h2]
        overlap = np.vdot(psi1, psi2)
        F[i, j] = np.abs(overlap)**2


In [6]:
#logF = np.log(F + 1e-12)
logF = F


In [7]:
import numpy as np
import plotly.graph_objects as go

H1, H2 = np.meshgrid(h_values, h_values)

z_min = float(np.min(logF))

fig = go.Figure()

# surface
fig.add_trace(
    go.Surface(
        x=H1, y=H2, z=logF,
        colorscale="Viridis",
        showscale=False,   # remove the “temperature legend”
        name="surface"
    )
)

fig.update_layout(
    title="Log Fidelity Surface (TFIM 4x4)",
    template="plotly_dark",
    margin=dict(l=0, r=0, b=0, t=40),
    scene=dict(
        xaxis_title="h",
        yaxis_title="h'",
        zaxis_title="-log F(h,h')",
        camera=dict(eye=dict(x=1.6, y=1.6, z=0.9))  # similar vibe to your elev/azim
    )
)

fig.show()


In [8]:
# save surface datapoints

np.savez_compressed("tfim_3x3_fidelity_surface.npz",
    h_values=h_values,
    logF=logF
)